# SciStream Fabric Example STAR ---- DALL 
### This exmaple test only the SciStream Protocol itself. According to the basic setups, this version may not need switches.
This notebook will setup a demo testbed for Scistream by utilizing Fabric resources. Will support QUIC implementation between channels. And will evaluate the performance through various streaming protocols (e.g., DASH).

Additional resources:
- [SciStream](https://scistream.github.io)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [SciStream Repo](https://github.com/scistream)
- [SciStream YouTube Presentations](https://youtu.be/IWKGjAa_TOA)
- [Topology Graph](https://drive.google.com/file/d/124xh4K15c7tUcaETXmj-JSPKcQLA_oTC/view?usp=sharing)

In [1]:
 import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Set your FABRIC PROJECT ID
os.environ['FABRIC_PROJECT_ID']='f4344c37-52e3-4d0c-a354-9311342ec109'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='cqy78_0038438951'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_bastion_key' #'id_rsa_fabric'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/work/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/work/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Basic FABRIC Slice Configuration

In [2]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [3]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
except Exception as e:
    print(f"Error: {e}")

Available Resources: Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
UCSD        10  320/320  2560/2560  116400/116400  635/635                 2/2                             4/4                            16/16               4/4               6/6
SALT         6  128/192  1280/1536  60580/60600    377/381                 2/2                             2/2                            10/10               2/2               3/3
UTAH        10  108/320  1734/2560  116218/116400  613/635                 2/2                             4/4                            16/16               4/4               5/5
MAX         10  188/320  2094/2560  115974/116400  575/63

## Configure Slice Parameters

This section builds the experiment slice. TO BE CHANGED, according to the code. this setup may change.

Note: Fabric did not support modifying resources after create slides. 



In [3]:
# Slice 
slice_name = 'SciStream-STAR-DALL'

# Switches --- This version may not need any switch
# s_name = "s1" 

# switch_cores = 2
# switch_ram = 8
# switch_disk = 100

# Hosts
h1_name = "h1" # consider as Facility 1 ProdApp
h2_name = "h2" # consider as Facility 1 ProdGN
h3_name = "h3" # consider as S2UC
h4_name = "h4" # consider as Facility 2 ConsGN
h5_name = "h5" # consider as Facility 2 ConsApp

h1_ip="192.168.0.1"
h2_ip="192.168.0.2" # prodGN - command: listener-ip: 192.168.0.2 s2-port 5000
h3_ip="192.168.0.3" # S2UC json file running prod and cons on 5000 but with different ip address
h4_ip="192.168.0.4" # consGN - command: listener-ip: 192.168.0.4 s2-port 5000
h5_ip="192.168.0.5"

h1_LAN_ip="172.16.0.1"
h2_LAN_ip="172.16.0.2"

h2_WAN_ip="172.16.1.1"
h4_WAN_ip="172.16.1.2"

h4_LAN_ip="172.16.2.1"
h5_LAN_ip="172.16.2.2"

host_cores = 32
host_ram = 128
host_disk = 10
# Sites
# Default set to TACC to support IPv4 and github clone...
# Should use proxy to access github once checked as IPv6 address. Or use other git platfrom, e.g., bibucket...
site_1 = 'STAR' # 'SALT' # 'NCSA' # s'WASH' # 'MAX' # 'STAR' # 'NCSA' # 'MAX' # 'UTAH' # 'DALL' # 'TACC' # 'MAX'
site_2 = 'DALL' # 'SALT' # 'NCSA' # s'WASH' # 'MAX' # 'STAR' # 'NCSA' # 'MAX' # 'UTAH' # 'DALL' # 'TACC' # 'MAX'

control_net_name = 'control_net' #no need for S2DS test only
prod_lan_net_name = 'prod_lan_net'
gn_net_name = 'GN_WAN_NET'
cons_lan_net_name = 'cons_lan_net'

net_pub_name = 'open_net'

# All node properties
#username = 'ubuntu'
image = 'default_ubuntu_20'
vlan = '1000'
#image_type = 'qcow2'




### Create the Slice

**This part is optional. Once the slice on, there is no need to have this block running.** \
In terms of the NIC usage. We prefer NIC_ConnectX_6 or NIC_Basic. They both provide 100 Gbps. However, NIC_ConnectX_5 still accepable. \
The network topology will be fixed and non-programmable after creating the slice. Be care of setup l2 or l3 networks. \
Network can be setup cross-site. A vlan is established based on your settings. The RTT is based on the physical distances. \
*In the newest FABRIC release, project manager may need to permissions from Admin to add cross-cite communication. Please attention.*

In [5]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node
#     switch1 = slice.add_node(name=s_name, site=site_2)
#     switch1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
#     switch1.set_image(image)
#     #[s1_iface_local] = s1.add_component(model='NIC_Basic', name="s1_local_nic").get_interfaces()
#     [s_iface_to_h1, s_iface_to_h2] = switch1.add_component(model='NIC_ConnectX_6', name="prod_switch_nic").get_interfaces()
#     s_iface_to_h1.set_vlan(vlan=vlan)
#     s_iface_to_h2.set_vlan(vlan=vlan)

#     s_iface_to_h3 = switch1.add_component(model='NIC_Basic', name="uc_local_nic").get_interfaces()[0]
#     s_iface_to_h3.set_vlan(vlan=vlan)
#     [s_iface_to_h4, s_iface_to_h5] = switch1.add_component(model='NIC_ConnectX_6', name="cons_switch_nic").get_interfaces()
#     s_iface_to_h4.set_vlan(vlan=vlan)
#     s_iface_to_h5.set_vlan(vlan=vlan)
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site_1)
    h1.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h1.set_image(image)
    h1_lan_iface = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()[0]
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site_1)
    h2.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h2.set_image(image)
    h2_lan_iface = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()[0]
    wan_iface1 = h2.add_component(model='NIC_Basic', name="h2_wan_nic").get_interfaces()[0]
 
    # Add host node h3 (no need for S2DS only test, but required for whole SciStream test)
    # h3 = slice.add_node(name=h3_name, site=site_2)
    # h3.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    # h3.set_image(image)
    # h3_iface = h3.add_component(model='NIC_Basic', name="h3_nic").get_interfaces()[0]
    
    # Add host node h4
    h4 = slice.add_node(name=h4_name, site=site_2)
    h4.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h4.set_image(image)
    h4_lan_iface = h4.add_component(model='NIC_Basic', name="h4_nic").get_interfaces()[0]
    wan_iface2 = h4.add_component(model='NIC_Basic', name="h4_wan_nic").get_interfaces()[0]
    
    # Add host node h5
    h5 = slice.add_node(name=h5_name, site=site_2)
    h5.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h5.set_image(image)
    h5_lan_iface = h5.add_component(model='NIC_Basic', name="h5_nic").get_interfaces()[0]
    

    #Add control panel networks 
    # control_net = slice.add_l2network(name=control_net_name, interfaces=[h1_iface,h2_iface, h3_iface, h4_iface,h5_iface])
    
    #Add Prod LAN
    prod_lan = slice.add_l2network(name=prod_lan_net_name, interfaces=[h1_lan_iface,h2_lan_iface])
    #Add Cons LAN
    cons_lan = slice.add_l2network(name=cons_lan_net_name, interfaces=[h4_lan_iface,h5_lan_iface])
    
    #Add GN WAN
    gn_wan = slice.add_l2network(name=gn_net_name, interfaces=[wan_iface1,wan_iface2])

    
    #Add L3 network for S2UC (May get error from Fabric)
    #net2 = slice.add_l3network(name=net_pub_name, interfaces=[h3_iface_pub], type='IPv4')
    
    #Submit Slice Request
    slice.submit() #(wait_progress=True)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    


---------------  ------------------------------------
Slice Name       SciStream-STAR-DALL
Slice ID         1b2edc03-af55-40b3-a220-6c2e6f7c3a4c
Slice State      StableOK
Lease End (UTC)  2022-08-19 04:15:13 +0000
---------------  ------------------------------------

Retry: 13, Time: 167 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP                           State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  --------------------------------------  -------  -------
b3419dee-9cb0-434b-bfbb-3f6787fbeba7  h1      STAR    star-w4.fabric-testbed.net       32    128      10  default_ubuntu_20  2001:400:a100:3030:f816:3eff:fe02:54ac  Active
6a8134e4-dce6-41ff-a905-02b25eff91af  h2      STAR    star-w5.fabric-testbed.net       32    128      10  default_ubuntu_20  2001:400:a100:3030:f816:3eff:fe01:cd47  Activ

# timeout needs to be extended

## Get the Slice

In [4]:
# ssh -F ~/work/fabric_config/ssh_config -i ~/work/id_rsa 
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"Node: {node.get_ssh_command()}")
except Exception as e:
    print(f"Exception: {e}")

Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:400:a100:3030:f816:3eff:fe02:54ac
Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:400:a100:3030:f816:3eff:fe01:cd47
Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:400:a100:3000:f816:3eff:fe70:900
Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:400:a100:3000:f816:3eff:fe30:ba29


## Configure Nodes
 ### This is just an exmple. Please wait for github to turn public and then run the script. Or the system will wait for cerdential.
 ### STEP 1: Setup controll net ---NO NEED

 ### STEP 2: Setup LAN net

In [7]:
try:
    h1 = slice.get_node(name=h1_name)        
    h1_os_iface = h1.get_interface(network_name=prod_lan_net_name)
    h1_os_iface.set_ip(ip=h1_LAN_ip, cidr="24")
    
    print(f"ifname: {h1_os_iface.get_os_interface()}")
    print(f"mac: {h1_os_iface.get_mac()}")
    stdout, stderr = h1.execute(f'sudo ip addr list {h1_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h1.execute(host_config_script_h1)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h2 = slice.get_node(name=h2_name)
    h2_os_iface = h2.get_interface(network_name=prod_lan_net_name)
    h2_os_iface.set_ip(ip=h2_LAN_ip, cidr="24")
 
    print(f"ifname: {h2_os_iface.get_os_interface()}")
    print(f"mac: {h2_os_iface.get_mac()}")
    stdout, stderr = h2.execute(f'sudo ip addr list {h2_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h2.execute(host_config_script_h2)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

try:
    h4 = slice.get_node(name=h4_name)
    h4_os_iface = h4.get_interface(network_name=cons_lan_net_name)
    h4_os_iface.set_ip(ip=h4_LAN_ip, cidr="24")

    print(f"ifname: {h4_os_iface.get_os_interface()}")
    print(f"mac: {h4_os_iface.get_mac()}")
    stdout, stderr = h4.execute(f'sudo ip addr list {h4_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h5 = slice.get_node(name=h5_name)
    h5_os_iface = h5.get_interface(network_name=cons_lan_net_name)
    h5_os_iface.set_ip(ip=h5_LAN_ip, cidr="24")

    print(f"ifname: {h5_os_iface.get_os_interface()}")
    print(f"mac: {h5_os_iface.get_mac()}")
    stdout, stderr = h5.execute(f'sudo ip addr list {h5_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

ifname: ens7
mac: 02:D2:B7:36:8C:AD
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:d2:b7:36:8c:ad brd ff:ff:ff:ff:ff:ff
    inet 172.16.0.1/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::d2:b7ff:fe36:8cad/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens7
mac: 02:19:33:50:93:2A
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:19:33:50:93:2a brd ff:ff:ff:ff:ff:ff
    inet 172.16.0.2/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::19:33ff:fe50:932a/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens7
mac: 02:44:2D:C0:D1:D3
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:44:2d:c0:d1:d3 brd ff:ff:ff:ff:ff:ff
    inet 172.16.2.1/24 scope global ens7
       valid_lft forever preferr

 ### STEP 3: Setup WAN net

In [8]:
try:
    h2 = slice.get_node(name=h2_name)
    h2_os_iface = h2.get_interface(network_name=gn_net_name)
    h2_os_iface.set_ip(ip=h2_WAN_ip, cidr="24")
 
    print(f"ifname: {h2_os_iface.get_os_interface()}")
    print(f"mac: {h2_os_iface.get_mac()}")
    stdout, stderr = h2.execute(f'sudo ip addr list {h2_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h2.execute(host_config_script_h2)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

try:
    h4 = slice.get_node(name=h4_name)
    h4_os_iface = h4.get_interface(network_name=gn_net_name)
    h4_os_iface.set_ip(ip=h4_WAN_ip, cidr="24")

    print(f"ifname: {h4_os_iface.get_os_interface()}")
    print(f"mac: {h4_os_iface.get_mac()}")
    stdout, stderr = h4.execute(f'sudo ip addr list {h4_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

ifname: ens8
mac: 06:8F:67:C2:A1:CC
4: ens8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 06:8f:67:c2:a1:cc brd ff:ff:ff:ff:ff:ff
    inet 172.16.1.1/24 scope global ens8
       valid_lft forever preferred_lft forever
    inet6 fe80::48f:67ff:fec2:a1cc/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens8
mac: 02:76:DD:CA:86:30
4: ens8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:76:dd:ca:86:30 brd ff:ff:ff:ff:ff:ff
    inet 172.16.1.2/24 scope global ens8
       valid_lft forever preferred_lft forever
    inet6 fe80::76:ddff:feca:8630/64 scope link 
       valid_lft forever preferred_lft forever




## TCP Tunning and iperf test

In [9]:
stdout, stderr = h1.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')
stdout, stderr = h2.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')

stdout, stderr = h4.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')
stdout, stderr = h5.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')

In [10]:
stdout, stderr = h1.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')

stdout, stderr = h2.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')


stdout, stderr = h4.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')

stdout, stderr = h5.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')


In [11]:
#ATTENTION: be aware of the NIC id!
#NEED SUDO! 
stdout, stderr = h1.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
print (stdout)
print (stderr)
stdout, stderr = h2.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
stdout, stderr = h2.execute('sudo ifconfig ens8 mtu 8900 up && sudo tc qdisc add dev ens8 root fq maxrate 30gbit')
print (stdout)
print (stderr)


stdout, stderr = h4.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
stdout, stderr = h4.execute('sudo ifconfig ens8 mtu 8900 up && sudo tc qdisc add dev ens8 root fq maxrate 30gbit')
print (stdout)
print (stderr)
stdout, stderr = h5.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
print (stdout)
print (stderr)

In [12]:

h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [13]:
#TCP TEST:
#Forward direction:
h1 = slice.get_node(name=h1_name) 
stdout, stderr = h1.execute(f'iperf -c 172.16.0.2 -P 32 -w 999M -t 30') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.0.2, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 35] local 172.16.0.1 port 34640 connected with 172.16.0.2 port 5001
[ 30] local 172.16.0.1 port 34634 connected with 172.16.0.2 port 5001
[ 31] local 172.16.0.1 port 34636 connected with 172.16.0.2 port 5001
[ 32] local 172.16.0.1 port 34638 connected with 172.16.0.2 port 5001
[  5] local 172.16.0.1 port 34580 connected with 172.16.0.2 port 5001
[  3] local 172.16.0.1 port 34582 connected with 172.16.0.2 port 5001
[  4] local 172.16.0.1 port 34578 connected with 172.16.0.2 port 5001
[  7] local 172.16.0.1 port 34588 connected with 172.16.0.2 port 5001
[  6] local 172.16.0.1 port 34584 connected with 172.16.0.2 port 5001
[  8] local 172.16.0.1 port 34586 connected with 172.16.0.2 port 5001
[  9] local 172.16.0.1 port 34592 connected with 172.16.0.2 port 5001
[

In [14]:
#Backward direction:
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute(f'iperf -c 172.16.1.1 -P 32 -w 999M -t 30') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.1.1, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 16] local 172.16.1.2 port 34908 connected with 172.16.1.1 port 5001
[ 24] local 172.16.1.2 port 34924 connected with 172.16.1.1 port 5001
[ 22] local 172.16.1.2 port 34922 connected with 172.16.1.1 port 5001
[ 23] local 172.16.1.2 port 34920 connected with 172.16.1.1 port 5001
[  5] local 172.16.1.2 port 34886 connected with 172.16.1.1 port 5001
[ 18] local 172.16.1.2 port 34912 connected with 172.16.1.1 port 5001
[  7] local 172.16.1.2 port 34890 connected with 172.16.1.1 port 5001
[ 11] local 172.16.1.2 port 34898 connected with 172.16.1.1 port 5001
[  6] local 172.16.1.2 port 34888 connected with 172.16.1.1 port 5001
[ 14] local 172.16.1.2 port 34902 connected with 172.16.1.1 port 5001
[  4] local 172.16.1.2 port 34882 connected with 172.16.1.1 port 5001
[

In [15]:
#UDP TEST:
#Forward direction
h1 = slice.get_node(name=h1_name) 
stdout, stderr = h1.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.0.2 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.0.2, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  6] local 172.16.0.1 port 60908 connected with 172.16.0.2 port 5001
[  4] local 172.16.0.1 port 49560 connected with 172.16.0.2 port 5001
[  7] local 172.16.0.1 port 36490 connected with 172.16.0.2 port 5001
[  3] local 172.16.0.1 port 37210 connected with 172.16.0.2 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0-10.0 sec  8.40 GBytes  7.21 Gbits/sec
[  3] Sent 1018873 datagrams
[  6]  0.0-10.0 sec  8.29 GBytes  7.12 Gbits/sec
[  6] Sent 1005740 datagrams
[  4]  0.0-10.0 sec  3.93 GBytes  3.38 Gbits/sec
[  4] Sent 477233 datagrams
[  7]  0.0-10.0 sec  5.97 GBytes  5.13 Gbits/sec
[  7] Sent 724753 datagrams
[SUM]  0.0-10.0 sec  26.6 GBytes  22.8 Gbits/sec
[SUM] Sent 3226599 data

In [16]:
#Backward direction
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.1.1 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.1.1, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  5] local 172.16.1.2 port 51725 connected with 172.16.1.1 port 5001
[  4] local 172.16.1.2 port 35155 connected with 172.16.1.1 port 5001
[  7] local 172.16.1.2 port 50806 connected with 172.16.1.1 port 5001
[  3] local 172.16.1.2 port 44382 connected with 172.16.1.1 port 5001
[ ID] Interval       Transfer     Bandwidth
[  4]  0.0-10.0 sec  5.48 GBytes  4.71 Gbits/sec
[  4] Sent 665465 datagrams
[  5]  0.0-10.0 sec  7.16 GBytes  6.15 Gbits/sec
[  5] Sent 868354 datagrams
[  7]  0.0-10.0 sec  7.39 GBytes  6.35 Gbits/sec
[  7] Sent 897220 datagrams
[  3]  0.0-10.0 sec  7.25 GBytes  6.23 Gbits/sec
[  3] Sent 879733 datagrams
[SUM]  0.0-10.0 sec  27.3 GBytes  23.4 Gbits/sec
[SUM] Sent 3310772 datagr

In [17]:
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [18]:
#TCP TEST:
#Forward direction
h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute(f'iperf -c 172.16.1.2 -P 32 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.1.2, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 33] local 172.16.1.1 port 57052 connected with 172.16.1.2 port 5001
[ 32] local 172.16.1.1 port 57048 connected with 172.16.1.2 port 5001
[ 31] local 172.16.1.1 port 57046 connected with 172.16.1.2 port 5001
[ 34] local 172.16.1.1 port 57050 connected with 172.16.1.2 port 5001
[  4] local 172.16.1.1 port 56992 connected with 172.16.1.2 port 5001
[  7] local 172.16.1.1 port 56996 connected with 172.16.1.2 port 5001
[  3] local 172.16.1.1 port 56990 connected with 172.16.1.2 port 5001
[  6] local 172.16.1.1 port 56994 connected with 172.16.1.2 port 5001
[  5] local 172.16.1.1 port 56998 connected with 172.16.1.2 port 5001
[  8] local 172.16.1.1 port 57000 connected with 172.16.1.2 port 5001
[ 10] local 172.16.1.1 port 57004 connected with 172.16.1.2 port 5001
[

In [19]:
#Backward direction
h5 = slice.get_node(name=h5_name) 
stdout, stderr = h5.execute(f'iperf -c 172.16.2.1 -P 32 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.2.1, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 35] local 172.16.2.2 port 38298 connected with 172.16.2.1 port 5001
[ 28] local 172.16.2.2 port 38292 connected with 172.16.2.1 port 5001
[ 33] local 172.16.2.2 port 38294 connected with 172.16.2.1 port 5001
[ 34] local 172.16.2.2 port 38296 connected with 172.16.2.1 port 5001
[  4] local 172.16.2.2 port 38236 connected with 172.16.2.1 port 5001
[  5] local 172.16.2.2 port 38240 connected with 172.16.2.1 port 5001
[  3] local 172.16.2.2 port 38238 connected with 172.16.2.1 port 5001
[  6] local 172.16.2.2 port 38244 connected with 172.16.2.1 port 5001
[  7] local 172.16.2.2 port 38242 connected with 172.16.2.1 port 5001
[ 10] local 172.16.2.2 port 38248 connected with 172.16.2.1 port 5001
[ 11] local 172.16.2.2 port 38250 connected with 172.16.2.1 port 5001
[

In [20]:
#UDP TEST:
#Forward direction
h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.1.2 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.1.2, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  3] local 172.16.1.1 port 33511 connected with 172.16.1.2 port 5001
[  4] local 172.16.1.1 port 53507 connected with 172.16.1.2 port 5001
[  5] local 172.16.1.1 port 41525 connected with 172.16.1.2 port 5001
[  6] local 172.16.1.1 port 38250 connected with 172.16.1.2 port 5001
[ ID] Interval       Transfer     Bandwidth
[  6]  0.0-10.0 sec  8.12 GBytes  6.98 Gbits/sec
[  6] Sent 985407 datagrams
[  4]  0.0-10.0 sec  1.54 GBytes  1.32 Gbits/sec
[  4] Sent 186987 datagrams
[  5]  0.0-10.0 sec  7.99 GBytes  6.86 Gbits/sec
[  5] Sent 969052 datagrams
[  3]  0.0-10.0 sec  8.39 GBytes  7.21 Gbits/sec
[  3] Sent 1018457 datagrams
[SUM]  0.0-10.0 sec  26.0 GBytes  22.4 Gbits/sec
[SUM] Sent 3159903 datag

In [21]:
#Backward direction
h5 = slice.get_node(name=h5_name) 
stdout, stderr = h5.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.2.1 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.2.1, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  6] local 172.16.2.2 port 47556 connected with 172.16.2.1 port 5001
[  3] local 172.16.2.2 port 40804 connected with 172.16.2.1 port 5001
[  4] local 172.16.2.2 port 49769 connected with 172.16.2.1 port 5001
[  5] local 172.16.2.2 port 42163 connected with 172.16.2.1 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0-10.0 sec  5.02 GBytes  4.31 Gbits/sec
[  3] Sent 608686 datagrams
[  6]  0.0-10.0 sec  3.33 GBytes  2.86 Gbits/sec
[  6] Sent 404360 datagrams
[  4]  0.0-10.0 sec  9.28 GBytes  7.98 Gbits/sec
[  4] Sent 1126767 datagrams
[  5]  0.0-10.0 sec  9.18 GBytes  7.89 Gbits/sec
[  5] Sent 1114366 datagrams
[SUM]  0.0-10.0 sec  26.8 GBytes  23.0 Gbits/sec
[SUM] Sent 3254179 data

In [5]:
h1 = slice.get_node(name=h1_name) 
stdout, stderr = h1.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [6]:
#TCP TEST:
#Backward direction
h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute(f'iperf -c 172.16.0.1 -P 32 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.0.1, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 30] local 172.16.0.2 port 36048 connected with 172.16.0.1 port 5001
[ 31] local 172.16.0.2 port 36040 connected with 172.16.0.1 port 5001
[ 35] local 172.16.0.2 port 36044 connected with 172.16.0.1 port 5001
[ 33] local 172.16.0.2 port 36046 connected with 172.16.0.1 port 5001
[  4] local 172.16.0.2 port 35986 connected with 172.16.0.1 port 5001
[  3] local 172.16.0.2 port 35988 connected with 172.16.0.1 port 5001
[  5] local 172.16.0.2 port 35990 connected with 172.16.0.1 port 5001
[  7] local 172.16.0.2 port 35992 connected with 172.16.0.1 port 5001
[  9] local 172.16.0.2 port 35998 connected with 172.16.0.1 port 5001
[  6] local 172.16.0.2 port 35994 connected with 172.16.0.1 port 5001
[ 10] local 172.16.0.2 port 35996 connected with 172.16.0.1 port 5001
[

In [7]:
#UDP TEST:
#Backward direction
h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.0.1 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.0.1, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  5] local 172.16.0.2 port 48173 connected with 172.16.0.1 port 5001
[  3] local 172.16.0.2 port 39252 connected with 172.16.0.1 port 5001
[  4] local 172.16.0.2 port 37692 connected with 172.16.0.1 port 5001
[  6] local 172.16.0.2 port 42541 connected with 172.16.0.1 port 5001
[ ID] Interval       Transfer     Bandwidth
[  6]  0.0-10.0 sec  7.84 GBytes  6.73 Gbits/sec
[  6] Sent 950955 datagrams
[  5]  0.0-10.0 sec  6.56 GBytes  5.63 Gbits/sec
[  5] Sent 796052 datagrams
[  3]  0.0-10.0 sec  3.03 GBytes  2.60 Gbits/sec
[  3] Sent 367387 datagrams
[  4]  0.0-10.0 sec  7.88 GBytes  6.77 Gbits/sec
[  4] Sent 956301 datagrams
[SUM]  0.0-10.0 sec  25.3 GBytes  21.7 Gbits/sec
[SUM] Sent 3070695 datagr

In [8]:
h5 = slice.get_node(name=h5_name) 
stdout, stderr = h5.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [9]:
#Backward direction:
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute(f'iperf -c 172.16.2.2 -P 32 -w 999M -t 30') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.2.2, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 33] local 172.16.2.1 port 57200 connected with 172.16.2.2 port 5001
[ 25] local 172.16.2.1 port 57192 connected with 172.16.2.2 port 5001
[ 30] local 172.16.2.1 port 57194 connected with 172.16.2.2 port 5001
[ 32] local 172.16.2.1 port 57196 connected with 172.16.2.2 port 5001
[  4] local 172.16.2.1 port 57138 connected with 172.16.2.2 port 5001
[  3] local 172.16.2.1 port 57142 connected with 172.16.2.2 port 5001
[ 11] local 172.16.2.1 port 57146 connected with 172.16.2.2 port 5001
[  6] local 172.16.2.1 port 57144 connected with 172.16.2.2 port 5001
[ 12] local 172.16.2.1 port 57150 connected with 172.16.2.2 port 5001
[ 13] local 172.16.2.1 port 57152 connected with 172.16.2.2 port 5001
[  5] local 172.16.2.1 port 57140 connected with 172.16.2.2 port 5001
[

In [10]:
#Forward direction
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.2.2 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.2.2, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  7] local 172.16.2.1 port 37666 connected with 172.16.2.2 port 5001
[  3] local 172.16.2.1 port 47367 connected with 172.16.2.2 port 5001
[  4] local 172.16.2.1 port 38501 connected with 172.16.2.2 port 5001
[  5] local 172.16.2.1 port 44805 connected with 172.16.2.2 port 5001
[ ID] Interval       Transfer     Bandwidth
[  7]  0.0-10.0 sec  8.85 GBytes  7.60 Gbits/sec
[  7] Sent 1074221 datagrams
[  3]  0.0-10.0 sec  5.41 GBytes  4.65 Gbits/sec
[  3] Sent 656764 datagrams
[  5]  0.0-10.0 sec  8.79 GBytes  7.55 Gbits/sec
[  5] Sent 1066246 datagrams
[  4]  0.0-10.0 sec  7.29 GBytes  6.26 Gbits/sec
[  4] Sent 884485 datagrams
[SUM]  0.0-10.0 sec  30.3 GBytes  26.1 Gbits/sec
[SUM] Sent 3681716 data

# Upload File and Execute

In [22]:
#upload_file(local_file_path, remote_file_path, retry=3, retry_interval=10)
# FILE 1: scistream source code

try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [23]:
#FILE 2: setup.sh
try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../setup.sh',"setup.sh", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../setup.sh',"setup.sh", retry=3, retry_interval=10)
   
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../setup.sh',"setup.sh", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../setup.sh',"setup.sh", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [24]:
#Optional file: QUIC proxy exmaple
try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
   
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [25]:
#Optional file: QUIC tunnel example
try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    h3 = slice.get_node(name=h3_name)
    h3.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

Error: Node not found: h3


Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 739, in get_node
    return Node.get_node(self, self.get_fim_topology().nodes[name])
  File "/opt/conda/lib/python3.9/site-packages/fim/view_only_dict.py", line 19, in __getitem__
    return self.__dict.__getitem__(k)
KeyError: 'h3'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_94/2642736187.py", line 8, in <module>
    h3 = slice.get_node(name=h3_name)
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 742, in get_node
    raise Exception(f"Node not found: {name}")
Exception: Node not found: h3


In [26]:
#Optional file: quic distribuation - quic-go version...
try:
    slice = fablib.get_slice(name=slice_name)
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    h3 = slice.get_node(name=h3_name)
    h3.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

Error: Node not found: h3


Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 739, in get_node
    return Node.get_node(self, self.get_fim_topology().nodes[name])
  File "/opt/conda/lib/python3.9/site-packages/fim/view_only_dict.py", line 19, in __getitem__
    return self.__dict.__getitem__(k)
KeyError: 'h3'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_94/1409364764.py", line 9, in <module>
    h3 = slice.get_node(name=h3_name)
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 742, in get_node
    raise Exception(f"Node not found: {name}")
Exception: Node not found: h3


In [27]:
#FILE 3: mona demo, only upload on h1 and h5
# try:
#     slice = fablib.get_slice(name=slice_name)
#     h1 = slice.get_node(name=h1_name)
#     h1.upload_file('mona-demo.zip',"mona-demo.zip", retry=3, retry_interval=10)
    
#     h5 = slice.get_node(name=h5_name)
#     h5.upload_file('mona-demo.zip',"mona-demo.zip", retry=3, retry_interval=10)
    
#     #stdout, stderr = h3.execute(host_config_script_h3)
#     #print("stdout: {}".format(stdout))
# except Exception as e:
#     print(f"Error: {e}")
#     traceback.print_exc()

In [28]:
#Optional file: tls certs for public usage. Ignore if test-only 
# try:
#     slice = fablib.get_slice(name=slice_name)
#     h1 = slice.get_node(name=h1_name)
#     h1.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
    
#     h2 = slice.get_node(name=h2_name)
#     h2.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
#     h3 = slice.get_node(name=h3_name)
#     h3.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
#     h4 = slice.get_node(name=h4_name)
#     h4.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
#     h5 = slice.get_node(name=h5_name)
#     h5.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
    
#     #stdout, stderr = h3.execute(host_config_script_h3)
#     #print("stdout: {}".format(stdout))
# except Exception as e:
#     print(f"Error: {e}")
#     traceback.print_exc()

In [29]:
#Optional download file: dumpfile...
# try:
#     slice = fablib.get_slice(name=slice_name)
#     h1 = slice.get_node(name=h1_name)
#     h1.download_file("dumpFile/prodGn_LAN.cap",'prodGn_LAN.cap', retry=3, retry_interval=10)
#     h2 = slice.get_node(name=h2_name)
#     h2.download_file("dumpFile/prodGn_WAN.cap",'prodGn_WAN.cap', retry=3, retry_interval=10)
#     h4 = slice.get_node(name=h4_name)
#     h4.download_file("dumpFile/consGn_WAN.cap",'consGn_WAN.cap', retry=3, retry_interval=10)
#     h5 = slice.get_node(name=h5_name)
#     h5.download_file("dumpFile/consGn_LAN.cap",'consGn_LAN.cap', retry=3, retry_interval=10)
#     h5 = slice.get_node(name=h5_name)
#     h5.download_file("dumpFile/ssl_log.txt",'aioquic/ssl_log.txt', retry=3, retry_interval=10)
# except Exception as e:
#     print(f"Error: {e}")
#     traceback.print_exc()

## Configure Nodes ----VERBOSE


In [30]:
master_script = "bash setup.sh"
try:
    h1 = slice.get_node(name=h1_name)        
    stdout, stderr = h1.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h1...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h2 = slice.get_node(name=h2_name)
    stdout, stderr = h2.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h2...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h4 = slice.get_node(name=h4_name) 
    stdout, stderr = h4.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h4...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h5 = slice.get_node(name=h5_name) 
    stdout, stderr = h5.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h5...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

done h1...
done h2...
done h4...
done h5...


## Below are some optinal operations for the slice

### Extend the slice reservation time

In [31]:
# import datetime

# #Set end host to now plus 1 day
# end_date = (datetime.datetime.utcnow() + datetime.timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S")

# try:
#     slice = fablib.get_slice(name=slice_name)

#     slice.renew(end_date)
# except Exception as e:
#     print(f"Exception: {e}")

### Check New Lease End Date

In [32]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     print(f"Lease End         : {slice.get_lease_end()}")
       
# except Exception as e:
#     print(f"Exception: {e}")

### DANGER ZONE --------*-------- Delete Slice

In [33]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Fail: {e}")
#     traceback.print_exc()